# Tesser inductive generalization modeling

In [1]:
# main_induc
import numpy as np
import scipy.spatial.distance as spsd
from scipy.signal import find_peaks_cwt as find_peak
import numpy.linalg as la
import matplotlib.pyplot as plt
import pandas as pd
import make_data as mkdt
import learning as ln
import sys
sys.path.append ('/home/rodrigo/Documents/preston_labs/tesser_successor/')

from tesser import network

path = '/home/rodrigo/Dropbox/tesser_successor/Data/'

In [2]:
'''
    data = dictionary containing structured learning data
    dl = list of directories of structured learning data serving as keys for 'data'
    induc_data = dictionary containing inductive data
    induc_list = list of directories of inductive data serving as keys for 'induc_data'
    
'''

sr_data, dl, induc_data, idl = mkdt.get_data(path)

In [3]:
# Print the number of participants in the data set and a list of subject names
print("There are %d subjects in this data set"% (len(idl)))
print()
for x in range(len(idl)):
    print('Subject %s is named %s'% (x,idl[x][:17]))

There are 37 subjects in this data set

Subject 0 is named tesserScan_100_B/
Subject 1 is named tesserScan_101_E/
Subject 2 is named tesserScan_102_S/
Subject 3 is named tesserScan_103_S/
Subject 4 is named tesserScan_104_J/
Subject 5 is named tesserScan_105_M/
Subject 6 is named tesserScan_106_H/
Subject 7 is named tesserScan_107_B/
Subject 8 is named tesserScan_108_M/
Subject 9 is named tesserScan_109_M/
Subject 10 is named tesserScan_110_AC
Subject 11 is named tesserScan_111_YM
Subject 12 is named tesserScan_112_JG
Subject 13 is named tesserScan_113_N/
Subject 14 is named tesserScan_114_SS
Subject 15 is named tesserScan_115_JG
Subject 16 is named tesserScan_116_GS
Subject 17 is named tesserScan_117_DC
Subject 18 is named tesserScan_119_SS
Subject 19 is named tesserScan_120_AS
Subject 20 is named tesserScan_121_TM
Subject 21 is named tesserScan_122_GL
Subject 22 is named tesserScan_123_AC
Subject 23 is named tesserScan_124_LO
Subject 24 is named tesserScan_125_DS
Subject 25 is named 

In [3]:
# Minor function to remove zeros in matrices with missing objects
def fix(matrix):
    new_matrix = matrix
    new_matrix[new_matrix == 0] = 0.0000001
    return new_matrix
# Transposing a matrix pattern to check for symmetry
def trans(matrix):
    m = np.transpose(matrix)
    return m
# Create a matrix pattern based on directory
def make_M_Matrix(dir,gamma, alpha, M):  
    envsteps= ln.make_envstep(dir)
    M  = ln.SRclass_nathum_exp1(envsteps, gamma, alpha, M) #W, mem, total_episodes
    return M
# Based on subject number, function returns the index of directory where the subject data starts 
def initial_subj_run(num):
    count = []
    for i in range(0, len(dl), 11):
        count.append(i)
    return count[num]


Objectives:

2. write a function that can take an SR matrix and induction data, and return log likelihood of the data given that SR matrix
To calculate log likelihood for a trial, you’ll need:
1. the cue item on that trial (i.e., the position of that item in the SR matrix)
2. the two choice items
3. which item was chosen
4. the SR matrix
From there, can calculate the probability using what’s known as the Luce choice rule. This is a more complete version of the equation I mentioned in the other channel:
P(choosing B|cued with A) = SR(A,B) / (SR(A,B) + SR(A,C))
P(choosing C|cued with A) = 1 - P(choosing B|cued with A)
You’ll want to save out the probability for the choice that the participant actually made on that trial.
Then, to get the log likelihood over all trials, add up the natural log of each the choice probability for each trial:
log(likelihood) = log(P(trial1)) + log(P(trial2)) + … + log(P(trialN))

In [4]:
print(idl[0])
induc_data[idl[0]].head(5)

tesserScan_100_B/tesserScan_100_B_InductGen.txt


,SubjNum,TrialNum,QuestType,Environment,CueNum,CueObject,Opt1Num,Option1,Opt2Num,Option2,Resp,Acc,RT
0,100,2,Bound2,desert,4,object_12.jpg,10,object_2.jpg,19,object_4.jpg,2,0,2.974157
1,100,3,Prim,desert,5,object_30.jpg,6,object_18.jpg,2,object_23.jpg,1,1,3.190848
2,100,4,Prim,ocean,2,object_23.jpg,9,object_35.jpg,19,object_4.jpg,2,1,3.373906
3,100,5,Prim,ocean,1,object_6.jpg,19,object_4.jpg,13,object_1.jpg,2,0,3.094699
4,100,6,Prim,forest,15,object_9.jpg,16,object_28.jpg,1,object_6.jpg,2,0,4.279663


In [6]:
sr_data[dl[0]].head(5)

,SubjNum,run,trial,objnum,file,orientnam,orientnum,resp,respnum,acc,rt
0,100,1,1,10,object_2.jpg,cor,1,c,1,1,1.426618
1,100,1,2,5,object_30.jpg,cor,1,c,1,1,0.759233
2,100,1,3,4,object_12.jpg,cor,1,c,1,1,0.866177
3,100,1,4,3,object_7.jpg,cor,1,c,1,1,0.712731
4,100,1,5,1,object_6.jpg,cor,1,c,1,1,0.654236


In [7]:
#from Demetriush Rowland
def pBGivenA (A, B, C, SR):
    return SR[A][B] / (SR[A][B] + SR[A][C])

def pCGivenA (A, B, C, SR):
    return 1 - pBGivenA (A, B, C, SR)

def likelihood (cue, opt1, opt2, response, SR):
    if response == 0:
        return pBGivenA (cue, opt1, opt2, SR)
    if response == 1:
        return pCGivenA (cue, opt1, opt2, SR)

In [9]:
def once(subj_num, alpha, gamma):
    a = initial_subj_run(subj_num)
    M = np.zeros([21,21])
    for run in range(0,11):
        envstep = ln.make_envstep (a + run)
        M += np.array (ln.SRclass_nathum_exp1 (envstep, gamma, alpha, np.copy(M)))
    return M   

def get_log_likelihood(subj_num, alpha, gamma):
    SR = once(subj_num, alpha, gamma)
    cue_sequence, opt1_sequence, opt2_sequence, response_sequence = ln.get_induc_data(subj_num)
    num_trials = len (cue_sequence)
    log_likelihood = 0
    for trial_num in range (num_trials):
        try:
            cue_num, opt1_num, opt2_num, response_num = int(cue_sequence[trial_num]) - 1, 
            int(opt1_sequence[trial_num]) - 1, int(opt2_sequence[trial_num]) - 1,
            int(response_sequence[trial_num]) - 1
            trial_probability = likelihood (cue_num, opt1_num, opt2_num, response_num, SR)
            log_likelihood += np.log (trial_probability)
        except ValueError:
            pass
    
    return log_likelihood


def main_log(subj_num):
    h = 10e-3
    alpha_max, gamma_max = 0.0, 0.0
    alpha, gamma = h, h
    log_likelihood_max = get_log_likelihood (subj_num, alpha, gamma)
    log_likelihoods = []
    
    
    while alpha <= 1:
        gamma = h
        while gamma <= 1:
            log_likelihood = get_log_likelihood(subj_num, alpha, gamma)
            log_likelihoods.append (log_likelihood)
            if log_likelihood > log_likelihood_max:
                alpha_max, gamma_max = alpha, gamma
            gamma += h
        alpha += h
    
    print(alpha_max, gamma_max)

main_log(1)

0.9900000000000007 0.9900000000000007
